In [ ]:
def check_for_proximal_windfarm(idx_2d, distance_to_windfarm = 3000):
    # Assuming your iris cube is named 'monthly_cube'
    # Get the coordinates of the center of the grid cell using the indices provided
    projection_y_coord = one_ts.coord('projection_y_coordinate').points[idx_2d[0]]
    projection_x_coord = one_ts.coord('projection_x_coordinate').points[idx_2d[1]]

    # Convert BNG coordinates of the grid cell to latitude and longitude
    lon_center, lat_center = bng_to_latlon(projection_x_coord, projection_y_coord)

    # Load your wind farms dataset containing 'X-coordinate' and 'Y-coordinate' columns in BNG
    # Assuming your wind farms dataset is named 'wind_farms_df'
    # Iterate over each wind farm location
    for index, row in windfarms.iterrows():
        # Convert BNG coordinates of wind farm location to latitude and longitude
        #lon_farm, lat_farm = bng_to_latlon(row['X-coordinate'], row['Y-coordinate'])
        # Calculate distance between wind farm location and the center of the grid cell
        distance = calculate_distance(lon_center, lat_center, row['lon'], row['lat'])

        # Check if the distance is within 3km
        if distance <= distance_to_windfarm:  # 3km in meters
            print(f"Wind farm at index {index} is within 3km of the grid cell.")
    print("Finished checking for close windfarms")


In [32]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import Proj, transform, Geod
import pandas as pd

windfarms = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/NIMROD/windfarms.csv', encoding='latin1')
windfarms =windfarms[windfarms['Technology Type']=='Wind Onshore']
windfarms = windfarms[(windfarms['Development Status'] == 'Operational') | (windfarms['Development Status'] == 'Decommissioned')]

# Read shapefile of UK outline
uk_outline = gpd.read_file('/nfs/a319/gy17m2a/PhD/datadir/SpatialData/UK_shp/GBR_adm0.shp')

# Define a function to convert BNG coordinates to latitude and longitude
def bng_to_latlon(x, y):
    in_proj = Proj(init='epsg:27700')  # British National Grid projection
    out_proj = Proj(init='epsg:4326')  # WGS84 (latitude and longitude) projection
    lon, lat = transform(in_proj, out_proj, x, y)
    return lon, lat

# # Read DataFrame with BNG coordinates
df = windfarms

# # Convert BNG coordinates to latitude and longitude
df['lon'], df['lat'] = zip(*df.apply(lambda row: bng_to_latlon(row['X-coordinate'], row['Y-coordinate']), axis=1))

# # Convert DataFrame to GeoDataFrame
# geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]
# gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# # Plot the UK outline and points
# fig, ax = plt.subplots(figsize=(10, 10))
# uk_outline.plot(ax=ax, color='lightgrey', edgecolor='black')
# gdf.plot(ax=ax, color='red', markersize=5)
# plt.xlabel('Longitude')
# plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_181023/1395697060.py", line 25, in <module>
    df['lon'], df['lat'] = zip(*df.apply(lambda row: bng_to_latlon(row['X-coordinate'], row['Y-coordinate']), axis=1))
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pandas/core/frame.py", line 8740, in apply
    return op.apply()
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pandas/core/apply.py", line 688, in apply
    return self.apply_standard()
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pandas/core/apply.py", line 812, in apply_standard
    results, res_index = self.apply_series_generator()
  File "/nfs/a321/gy17m2a/anaconda_install/anacon

TypeError: object of type 'NoneType' has no len()

In [1]:
import iris
import glob
import datetime as datetime
import sys
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyproj import Transformer
import itertools
from scipy import spatial
import numpy.ma as ma
import time
import math
import tilemapbase

OpenBLAS blas_thread_init: pthread_create failed for thread 30 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 2061748 max
OpenBLAS blas_thread_init: pthread_create failed for thread 31 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 4096 current, 2061748 max


KeyboardInterrupt: 

In [ ]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

# Create path to files containing functions
root_fp = "/nfs/a319/gy17m2a/PhD/"
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *
gb_gdf = create_gb_outline({'init' :'epsg:3857'})

In [ ]:
# Function to calculate distance between two points
def calculate_distance(lon1, lat1, lon2, lat2):
    geod = Geod(ellps='WGS84')
    _, _, distance = geod.inv(lon1, lat1, lon2, lat2)
    return distance

In [ ]:
# Define the directory and create a sorted list of file paths
year = 2012
radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
file_list = glob.glob(radardir + "*.nc")
sorted_list = sorted(file_list)

# Load the cubes
cubes = iris.load(sorted_list[0])

# Concatenate into a single cube
one_day = cubes.concatenate_cube()

In [ ]:
one_ts = one_day[1,:,:]

In [ ]:
for gauge_num in range(348,len(tbo_vals)):
    print(gauge_num)
    gauge = tbo_vals.iloc[gauge_num]

    # closest_point, idx_2d = find_position_obs(one_ts,gauge1['Lat'], gauge1['Lon'], plot=True)
    closest_point, idx_2d = find_position_obs_nomasking(one_ts,gauge['Lat'], gauge['Lon'], plot=False)
    check_for_proximal_windfarm(idx_2d, 5000)
    end=time.time()


In [ ]:
for gauge_num in range(348,len(tbo_vals)):
    print(gauge_num)
    gauge = tbo_vals.iloc[gauge_num]

    # closest_point, idx_2d = find_position_obs(one_ts,gauge1['Lat'], gauge1['Lon'], plot=True)
    closest_point, idx_2d = find_position_obs_nomasking(one_ts,gauge['Lat'], gauge['Lon'], plot=False)
    check_for_proximal_windfarm(idx_2d, 5000)
    end=time.time()


20
Finished checking for close windfarms
21
Finished checking for close windfarms
22
Finished checking for close windfarms
23
Finished checking for close windfarms
24
Finished checking for close windfarms
25
Finished checking for close windfarms
26
Finished checking for close windfarms
27
Wind farm at index 2755 is within 3km of the grid cell.
Wind farm at index 3074 is within 3km of the grid cell.
Finished checking for close windfarms
28
Finished checking for close windfarms
29
Finished checking for close windfarms
30
Finished checking for close windfarms
31
Finished checking for close windfarms
32
Finished checking for close windfarms
33
Finished checking for close windfarms
34
Finished checking for close windfarms
35
Wind farm at index 3074 is within 3km of the grid cell.
Finished checking for close windfarms
36
Wind farm at index 2095 is within 3km of the grid cell.
Wind farm at index 2421 is within 3km of the grid cell.
Wind farm at index 2763 is within 3km of the grid cell.
Wind 

### At each grid cell with a gauge in it, check the NIMROD time series

In [62]:
big_list = []
for year in range(2006, 2018):
    if year != 2017:
        print(year)
        radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
        file_list = glob.glob(radardir + "*.nc")
        sorted_list = sorted(file_list)
        cubes = iris.load(sorted_list)
        common_metadata = cubes[0].metadata
        for cube in cubes:
            cube.metadata = common_metadata  
        monthly_cube = cubes.concatenate_cube()
        ts_one_location = monthly_cube[:,825,891]
        print(ts_one_location.shape)
        print(np.nanmax(ts_one_location.data))
        
        


2006
(24157,)


Exception ignored in: <function CFReader.__del__ at 0x7f7fe5fe6680>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


48.3125
2007
(25120,)


/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+-----------+-----------+-----------+
| Package | client    | scheduler | workers   |
+---------+-----------+-----------+-----------+
| dask    | 2021.10.0 | 2022.02.0 | 2021.10.0 |
+---------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+-----------+-----------+-----------+
| Package | client    | scheduler | workers   |
+---------+-----------+-----------+-----------+
| dask    | 2021.10.0 | 2022.02.0 | 2021.10.0 |
+---------+-----------+-----------+-----------+
  warnings.war

42.46875
2008
(24853,)


...

103.96875
2015
(25374,)


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


25.0625
2016
(26364,)


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


42.46875


In [41]:
ts_one_location = monthly_cube[:,825,891]
np.nanmax(ts_one_location.data)

In [54]:
flattened_data = ts_one_location.data.flatten()

# # Calculate summary statistics for the selected grid cell
# mean_value = np.nanmean(flattened_data)
# std_value = np.nanstd(flattened_data)
# min_value = np.nanmin(flattened_data)
# max_value = np.nanmax(flattened_data)

# # Define threshold for outliers (e.g., values more than 3 standard deviations from the mean)
# threshold = 3
# lower_bound = mean_value - threshold * std_value
# upper_bound = mean_value + threshold * std_value

# # Identify outliers
# outliers = flattened_data[(flattened_data < lower_bound) | (flattened_data > upper_bound)]

In [ ]:
def max_value_for_cell(cell_data):
    return np.nanmax(cell_data.data)

# Map the function to each grid cell in parallel using Dask
max_values = monthly_cube.map_blocks(max_value_for_cell)

# Compute the maximum values
max_values_computed = max_values.compute()

In [59]:
print(ts_one_location.coord('time'))

DimCoord([2017-06-01 00:00:00, 2017-06-01 00:05:00, 2017-06-01 00:10:00, ...,
       2017-08-31 23:45:00, 2017-08-31 23:50:00, 2017-08-31 23:55:00], standard_name='time', calendar='gregorian', var_name='time')


In [56]:
# cubes = iris.load(sorted_list)
# monthly_cube = cubes.concatenate_cube()
# ts_one_location = monthly_cube[:,825,891]
np.nanmax(ts_one_location.data)

104.375

In [ ]:
# np.nanmax(monthly_cube[:,825,891].data)
# np.histogram(monthly_cube[:,825,891].data)

In [ ]:
# import iris.plot as iplt
# iplt.plot(ts_one_location)

In [98]:
# # Get tb0 values at each gauge
# tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# gauge1 = tbo_vals.iloc[1]

# # closest_point, idx_2d = find_position_obs(one_ts,gauge1['Lat'], gauge1['Lon'], plot=True)
# closest_point, idx_2d = find_position_obs_nomasking(one_ts,53.399100, -4.387940, plot=True)

In [12]:
### Read in a sample cube for finding the location of gauge in grid
sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070601_30mins.nc')[0][1,:,:]

# Get tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')

Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(1, sample_cube)

NameError: name 'find_position_obs' is not defined

In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
warnings.filterwarnings("ignore", category =UserWarning,)

from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
import iris.plot as iplt
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

# from Identify_Events_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)



In [19]:
import xarray as xr

def process_file(file_path):
    # Load the dataset using xarray
    ds = xr.open_dataset(file_path)
    
    # Extract the variable of interest (assuming it's named 'variable_name')
    this_grid_cell = ds['rain_rate_composite'].isel(projection_y_coordinate=70, projection_x_coordinate=405)
    
    # Compute the maximum value
    max_value = this_grid_cell.max().item()
    
    return max_value

# Define the directory and create a sorted list of file paths
file_paths = []
for year in range(2006, 2007):
    radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
    file_list = glob.glob(radardir + "*.nc")
    sorted_list = sorted(file_list)[:30]
    file_paths.extend(sorted_list)

# Process files in parallel
results = [process_file(file_path) for file_path in file_paths]

# Compute results
max_value = max(results)


Exception ignored in: <function CFReader.__del__ at 0x7f4c677fedd0>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


RuntimeError: can't start new thread

In [ ]:

def find_amax_indy_events_v1 (df, duration, Tb0):
    
    rainfall_events = find_independent_events(df, Tb0)
    max_val, max_df = find_max_for_this_duration(rainfall_events, duration = duration)
    return max_df

def find_amax_indy_events_v2 (df, duration, Tb0):
    
    rainfall_cores = find_rainfall_core(df, duration=duration, Tb0= Tb0)
    rainfall_events_expanded= []

    for rainfall_core in rainfall_cores:
        #print(len(rainfall_core))
        rainfall_core_after_search1 = search1(df, rainfall_core)
        #print(len(rainfall_core_after_search1))
        rainfall_core_after_search2 = search2(df, rainfall_core_after_search1)
        #print(len(rainfall_core_after_search2))
        rainfall_core_after_search3 = search3(df, rainfall_core_after_search2, Tb0= Tb0)
        #print(len(rainfall_core_after_search3))
        # If the event is not entirely dry 
        if len(rainfall_core_after_search3[rainfall_core_after_search3['precipitation (mm/hr)']>0.1]) >0:
            rainfall_events_expanded.append(rainfall_core_after_search3)
    
    return rainfall_events_expanded

def find_gauge_Tb0_and_location_in_grid (gauge_num, sample_cube):
    # Get data just for this gauge
    gauge1 = tbo_vals.iloc[gauge_num]
    # Find the interevent arrival time (Tb0)
    Tb0 = int(gauge1['Critical_interarrival_time'])
    # Find the coordinates of the cell containing this gauge
    closest_point, idx_2d = find_position_obs(sample_cube,gauge1['Lat'], gauge1['Lon'], plot=True)
    
    return Tb0, idx_2d

def read_model_data(em, yr, idx_2d):
    filename = f'/nfs/a319/gy17m2a/PhD/ProcessedData/TimeSeries/UKCP18_every30mins/2.2km/2002_2020/{em}_wholeyear/{yr}_maskedcube.nc'
    cube = iris.load(filename)[0]
    # This gauge
    data = cube[:,idx_2d[0],idx_2d[1]].data
    # Data as dataframe
    df= pd.DataFrame({'precipitation (mm/hr)':data})
    df['times'] = cube[:,idx_2d[0],idx_2d[1]].coord('time').units.num2date(cube.coord('time').points)
    # New precipitation accumulation column
    df['precipitation (mm)'] = df['precipitation (mm/hr)']/2            
    return df 




# # I think this was too check that numbers werent getting added at the wrong end to events in the windows
# def is_sequential(numbers):
#     return all(numbers[i] - numbers[i - 1] == 1 for i in range(1, len(numbers)))
# lens_v2 =[]

# Loop through gauges
for gauge_num in range(380,381):
    print(gauge_num)
    
    # Find the Tb0 and index of this gauge
    Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(gauge_num, sample_cube)
    
    # For each ensemble member
    for em in ['bc005']:

        # Make directory to store outputs
        #if not os.path.isdir(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option1"):
        #    os.makedirs(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option1")
        if not os.path.isdir(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option2"):
            os.makedirs(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option2")

        for yr in range(2008,2020):

            if not all(os.path.exists(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/{option}/{duration}hrs_{yr}_v1.csv") for option in ['Option1', 'Option2'] for duration in [0.5, 1, 2, 3, 6, 12, 24]):

                # Get data for this ensemble member and year, at the grid cell containing this gauge
                df =read_model_data(em, yr, idx_2d)

                # Loop through duration
                for duration in [0.5, 1, 2, 3, 6, 12, 24]:    
                    print(f"Finding the AMAX for {duration}hr events in em {em} for the year {yr}")
                    ###################
                    # Option 1
                    ###################
                    #if os.path.isfile(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option1/{duration}hrs_{yr}_v1.csv"):
                    #    print("Option 1 already exists")
                    #else:
                    #    event_v1 = find_amax_indy_events_v1(df, duration =duration, Tb0=Tb0)
                    #    lens_v1.append(len(event_v1))
                    #    if len(event_v1)>1:
                    #        event_v1.to_csv(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option1/{duration}hrs_{yr}_v1.csv")
                    #        print(f"Option 1: length with my method {len(event_v1)/2}, from {event_v1.index[0]}  to {event_v1.index[-1]} ")

                    ###################
                    # Option 2
                    ###################                
                    if os.path.isfile(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option2/{duration}hrs_{yr}_v2_part0.csv"):
                        print("Option 2 already exists")
                    else:
                        # Find event (might be more than one)
                        events_v2 = find_amax_indy_events_v2(df, duration =duration, Tb0=Tb0)
                        # For each of the events
                        for num, event in enumerate(events_v2):
                            if len(event)>1:
#                                 # Add length to list of lengths
#                                 lens_v2.append(len(event))
#                                 # Check if event index always increasing by one
#                                 if event.index.is_monotonic_increasing ==False:
#                                     print(f"monotonic: {duration}, {yr}")
#                                 # And check index is sequential
#                                 if not is_sequential(event.index):
#                                     print(f"not sequential {file}")
                                event.to_csv(f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option2/{duration}hrs_{yr}_v2_part{num}.csv")
                                # print(f"Option 2: length with RVH method {len(events_v2[num])/2}, from {events_v2[num].index[0]}  to {events_v2[num].index[-1]} ")

            else:
                print(f"all files already exist for {yr}")